In [15]:
# libs para processamento
import json

import geopandas as gpd
import pandas as pd

from hidromet.ANA import ANA
from hidromet.utils import dir_projeto
from shapely.geometry import Point
from hidromet import limpeza
from hidromet import modelos
from pathlib import Path

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
diretorio = dir_projeto()

# Obter inventário de postos da ANA

In [3]:
ana = ANA()
inventario_plu = ana.inventario(tipoest=2)
inventario_plu.loc[:, ["latitude", "longitude"]] = inventario_plu.loc[:, ["latitude", "longitude"]].apply(pd.to_numeric)

# Recortar postos dentro do contorno das bacias

## Tratamento dos contornos

In [4]:
contornos = diretorio.joinpath("contornos", "iguacu.shp")

gdf = gpd.read_file(contornos)
crs = gdf.crs

In [5]:
coords = [Point(x) for x in zip(inventario_plu.longitude,
                                inventario_plu.latitude)]

postos_plu_ana = gpd.GeoDataFrame(inventario_plu, geometry = coords, crs = crs)

## Postos da ANA dentro dos contornos

In [6]:
bacias = list()
for ix in gdf.index:
    geometria = gdf.loc[ix, 'geometry']
    bacia = gdf.loc[ix, 'bacia']
    recorte_postos_ana = postos_plu_ana.geometry.within(geometria)
    postos_ana_dentro = postos_plu_ana.loc[recorte_postos_ana]
    postos_ana_dentro = postos_ana_dentro.assign(bacia=bacia)

    bacias.append(postos_ana_dentro)



In [7]:
df_postos_bacias = pd.concat(bacias)

In [8]:
dir_postos = diretorio.joinpath("arquivos", "postos_plu_ana_smap.csv")
df_postos_bacias.to_csv(dir_postos)

## Obter séries de chuva por bacia

In [12]:
dir_prec = diretorio.joinpath("arquivos", "series-chuva-ana")
dir_prec.mkdir(parents=True, exist_ok=True)

bacias_gp = df_postos_bacias.groupby("bacia")
bacias = bacias_gp.groups.keys()

info_bacias = list()
for bacia in bacias:
    arquivo_bacia = dir_prec.joinpath(f"{bacia}.csv")

    print(f"Obtendo séries de chuva para postos na bacia {bacia}.")
    bacia_df = bacias_gp.get_group(bacia)
    bacia_df = limpeza.remover_codigos_duplicados(bacia_df)
    n_postos = len(bacia_df.index)

    chuvas_bacia = list()
    postos_sem_dado = list()
    postos_com_falha = list()
    postos_nao_representativos = list()
    postos_ok = list()
    for posto in bacia_df.index:
        df = ana.obter_chuva(cod_estacao=posto)
        df.index = pd.to_datetime(df.index)

        df = limpeza.remover_duplicados(df)
        df[posto] = limpeza.recortar_periodo(
            serie=df[posto], 
            inicio=pd.to_datetime("20000101"), 
            fim=pd.Timestamp('today')
            )
        df[posto] = limpeza.remover_outliers(df[posto], p=0.99)
        df[posto] = limpeza.substituir_negativos(df[posto])
        falhas = limpeza.contabilizar_falhas(df[posto])
        df = limpeza.remover_meses_nao_representativos(df)
        anos_disponiveis = limpeza.anos_disponiveis(df)
        n_dados = len(df)


        info_posto = modelos.Posto(
            latitude=bacia_df.loc[posto, 'latitude'],
            longitude=bacia_df.loc[posto, 'longitude'],
            codigo=posto,
            n_dados=n_dados
        )


        if not df.empty and falhas < 0.15 and anos_disponiveis > 8:
            postos_ok.append(info_posto)
            chuvas_bacia.append(df)
        elif df.empty:
            postos_sem_dado.append(info_posto)
        elif anos_disponiveis < 8: 
            postos_nao_representativos.append(info_posto)
        elif falhas > 0.15:
            postos_com_falha.append(info_posto)


    info_bacia = modelos.Bacia(
        serie_vazia=postos_sem_dado,
        nao_representativos=postos_nao_representativos,
        com_falhas=postos_com_falha,
        postos_ok=postos_ok,
        n_postos=n_postos,
        bacia=bacia
    )

    info_bacias.append(info_bacia)
    try:
        df_chuvas_bacia = pd.concat(chuvas_bacia, axis=1)
        df_chuvas_bacia.to_csv(arquivo_bacia)
    except (ValueError):
        print(
            f"Não há séries de precipitação disponíveis para os postos encontrados na bacia {bacia}"
        )

Obtendo séries de chuva para postos na bacia Iguaçú.


In [13]:
for info in info_bacias:
    arquivo_json = Path(dir_prec, f'coords_{info.bacia}.json')
    with open(arquivo_json, 'w') as f:
        json.dump(info.dict(), f)

In [16]:
postos_ok_bacia = info.postos_ok

In [17]:
for posto in postos_ok_bacia:
    print(posto)

latitude=-25.8756 longitude=-50.3894 codigo='2550069'
latitude=-25.3656 longitude=-51.4956 codigo='2551015'
latitude=-25.8075 longitude=-51.2886 codigo='2551054'
latitude=-25.2406 longitude=-51.9275 codigo='2551063'
latitude=-25.6617 longitude=-51.6622 codigo='2551069'
latitude=-25.7747 longitude=-52.1139 codigo='2552055'
latitude=-25.5086 longitude=-52.2292 codigo='2552061'
latitude=-25.6283 longitude=-52.6158 codigo='2552070'
latitude=-25.3939 longitude=-53.1039 codigo='2553062'
latitude=-26.1547 longitude=-49.3806 codigo='2649073'
latitude=-26.0192 longitude=-50.5925 codigo='2650027'
latitude=-26.0931 longitude=-51.8797 codigo='2651054'
latitude=-26.0094 longitude=-51.6647 codigo='2651055'
latitude=-26.0306 longitude=-51.1419 codigo='2651056'
latitude=-26.3867 longitude=-51.2719 codigo='2651057'
latitude=-26.1653 longitude=-51.2281 codigo='2651058'
latitude=-26.2281 longitude=-51.0803 codigo='2651059'
